# Racial Bias Analysis

### Setting up environment and loading Preprocessed Training/Test Data

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import sem
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
xTrain=pd.read_csv('xTrain.csv')
xTest=pd.read_csv('xTest.csv')
yTrain=pd.read_csv('yTrain.csv')
yTest=pd.read_csv('yTest.csv')
xTest.head()
xTrain = xTrain.drop(columns=['Unnamed: 0'])
xTest = xTest.drop(columns=['Unnamed: 0'])

### Creating test sets with only specific races for Bias Analysis

In [ ]:
frames = [xTest, yTest]
TestSet = pd.concat(frames, axis=1)
TestSet = TestSet.drop(columns=['Unnamed: 0'])
TestSet.head()

In [ ]:
TestWhite = TestSet[TestSet['ethnicity'] == 'WHITE']
TestBlack = TestSet[TestSet['ethnicity'] == 'BLACK/AFRICAN AMERICAN']
TestHisp = TestSet[TestSet['ethnicity'] == 'HISPANIC/LATINO']
TestAsian = TestSet[TestSet['ethnicity'] == 'ASIAN']

In [ ]:
xTestWhite = TestWhite.drop(columns=['ethnicity', 'hospital_expire_flag'])
yTestWhite = TestWhite['hospital_expire_flag'].reset_index()
yTestWhite = yTestWhite.drop(columns=['index'])
xTestBlack = TestBlack.drop(columns=['ethnicity', 'hospital_expire_flag'])
yTestBlack = TestBlack['hospital_expire_flag'].reset_index()
yTestBlack = yTestBlack.drop(columns=['index'])
xTestHisp = TestHisp.drop(columns=['ethnicity', 'hospital_expire_flag'])
yTestHisp = TestHisp['hospital_expire_flag'].reset_index()
yTestHisp = yTestHisp.drop(columns=['index'])
xTestAsian = TestAsian.drop(columns=['ethnicity', 'hospital_expire_flag'])
yTestAsian = TestAsian['hospital_expire_flag'].reset_index()
yTestAsian = yTestAsian.drop(columns=['index'])

Final edit to training and testing dataset

In [ ]:
xTrain=xTrain.drop(columns=['ethnicity'])
xTest=xTest.drop(columns=['ethnicity'])
yTrain=yTrain['hospital_expire_flag']
yTest=yTest['hospital_expire_flag']

In [ ]:
xTest.head()

In [ ]:
xTestWhite.head()

In [ ]:
xTrain.head()

## Calculating overall AUROC w/ 95% Confidence Intervals for Chosen Model

In [ ]:
model=RandomForestClassifier(n_estimators = 400, criterion='entropy', max_depth=20, min_samples_leaf=50, max_features=17, bootstrap=True, oob_score=True)
model.fit(xTrain, yTrain)
y_pred = model.predict_proba(xTest)

In [ ]:
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTest, y_pred[:,1])))

In [ ]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_pred), len(y_pred))
    if len(np.unique(yTest[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(yTest[indices], y_pred[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

In [ ]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

# Computing the lower and upper bound of the 90% confidence interval
# You can change the bounds percentiles to 0.025 and 0.975 to get
# a 95% confidence interval instead.
confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

### Calculating 95% CI for individual races for Chosen Model

#### For White Patients Set

In [ ]:
y_pred_white = model.predict_proba(xTestWhite)
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTestWhite, y_pred_white[:,1])))

In [ ]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_pred_white), len(y_pred_white))

    score = roc_auc_score(yTestWhite.to_numpy()[indices], y_pred_white[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

In [ ]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score on White patients: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

#### For Black Patient Set

In [ ]:
y_pred_black = model.predict_proba(xTestBlack)
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTestBlack, y_pred_black[:,1])))

In [ ]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_black), len(y_pred_black))

    score = roc_auc_score(yTestBlack.to_numpy()[indices], y_pred_black[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

In [ ]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score on Black patients: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

#### For Hispanic Patient Set

In [ ]:
y_pred_hispanic = model.predict_proba(xTestHisp)
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTestHisp, y_pred_hispanic[:,1])))

In [ ]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_hispanic), len(y_pred_hispanic))

    score = roc_auc_score(yTestHisp.to_numpy()[indices], y_pred_hispanic[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

In [ ]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()
confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score on Hispanic patients: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

#### For Asian Patient Set

In [ ]:
y_pred_asian = model.predict_proba(xTestAsian)
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTestAsian, y_pred_asian[:,1])))

In [ ]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_asian), len(y_pred_asian))

    score = roc_auc_score(yTestAsian.to_numpy()[indices], y_pred_asian[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

In [ ]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score on Asian patients: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

## Calculating overall AUROC w/ 95% Confidence Intervals for Logistic Regression Model

In [ ]:
xTrainLog = xTrain['sofa_24hours'].to_numpy().reshape(-1,1)
xTestLog = xTest['sofa_24hours'].to_numpy().reshape(-1,1)

In [ ]:
model=LogisticRegression()
model.fit(xTrainLog, yTrain)
y_pred = model.predict_proba(xTestLog)

In [ ]:
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTest, y_pred[:,1])))

In [ ]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_pred), len(y_pred))
    if len(np.unique(yTest[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(yTest[indices], y_pred[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

In [ ]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()
confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

### Selecting only SOFA score for individual race test dataset

In [ ]:
xTestWhite = xTestWhite['sofa_24hours'].to_numpy().reshape(-1,1)
xTestBlack = xTestBlack['sofa_24hours'].to_numpy().reshape(-1,1)
xTestHisp = xTestHisp['sofa_24hours'].to_numpy().reshape(-1,1)
xTestAsian = xTestAsian['sofa_24hours'].to_numpy().reshape(-1,1)

#### For White Patient Set

In [ ]:
y_pred_white = model.predict_proba(xTestWhite)
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTestWhite, y_pred_white[:,1])))

In [ ]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_white), len(y_pred_white))

    score = roc_auc_score(yTestWhite.to_numpy()[indices], y_pred_white[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

In [ ]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score on White patients: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

#### For Black Patient Set

In [ ]:
y_pred_black = model.predict_proba(xTestBlack)
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTestBlack, y_pred_black[:,1])))

In [ ]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_black), len(y_pred_black))

    score = roc_auc_score(yTestBlack.to_numpy()[indices], y_pred_black[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

In [ ]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score on Black patients: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

#### For Hispanic Patient Set

In [ ]:
y_pred_hispanic = model.predict_proba(xTestHisp)
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTestHisp, y_pred_hispanic[:,1])))

In [ ]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_hispanic), len(y_pred_hispanic))

    score = roc_auc_score(yTestHisp.to_numpy()[indices], y_pred_hispanic[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

In [ ]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()
confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score on Hispanic patients: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

#### For Asian Patient Set

In [ ]:
y_pred_asian = model.predict_proba(xTestAsian)
print("Original ROC area: {:0.3f}".format(roc_auc_score(yTestAsian, y_pred_asian[:,1])))

In [ ]:
n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_asian), len(y_pred_asian))

    score = roc_auc_score(yTestAsian.to_numpy()[indices], y_pred_asian[indices, 1])
    bootstrapped_scores.append(score)
    print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

In [ ]:
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score on Asian patients: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))